In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [4]:
def create_improved_model(input_dim):
    """
    Crear un modelo mejorado con mejores prácticas
    """
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim,
              kernel_initializer='glorot_uniform'),
        Dropout(0.3),
        Dense(32, activation='relu',
              kernel_initializer='glorot_uniform'),
        Dropout(0.3),
        Dense(1, activation='sigmoid',
              kernel_initializer='glorot_uniform')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Modelo"):
    """
    Entrenar y evaluar modelo
    """
    print(f"Entrenando {model_name}")
    
    # Callbacks para monitorear el entrenamiento
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=15, 
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=10, 
        min_lr=0.0001,
        verbose=1
    )
    
    # Entrenar modelo
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluar modelo
    print(f"\nEvluación {model_name}")
    
    # Predicciones
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Estadísticas de probabilidades
    print(f"Estadísticas de probabilidades:")
    print(f"  Min: {y_pred_prob.min():.6f}")
    print(f"  Max: {y_pred_prob.max():.6f}")
    print(f"  Mean: {y_pred_prob.mean():.6f}")
    print(f"  Std: {y_pred_prob.std():.6f}")
    print(f"  Mediana: {np.median(y_pred_prob):.6f}")
    
    # Distribución de predicciones
    print(f"\nDistribución de predicciones:")
    unique, counts = np.unique(y_pred, return_counts=True)
    for val, count in zip(unique, counts):
        label = "exitoso" if val == 1 else "no_exitoso"
        print(f"  {label}: {count}")
    
    # Matriz de confusión
    print(f"\nMatriz de confusión:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Reporte de clasificación
    print(f"\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=['no_exitoso', 'exitoso']))
    
    # Verificar pesos del modelo
    print(f"\nAnálisis de pesos:")
    for i, layer in enumerate(model.layers):
        if hasattr(layer, 'get_weights') and layer.get_weights():
            weights = layer.get_weights()[0]
            print(f"  Capa {i}: shape={weights.shape}, rango=[{weights.min():.6f}, {weights.max():.6f}]")
    
    return history, y_pred_prob

In [ ]:
# Cargar datos
df = pd.read_csv("../Datos2011Limipios.csv")
print(f"Datos originales: {df.shape}")
print(f"Distribución original:\n{df['resultado'].value_counts()}")

Datos originales: (494094, 29)
Distribución original:
resultado
no_exitoso    476685
exitoso        17409
Name: count, dtype: int64


In [7]:
def balancear_dataframe_simple(df, columna_resultado):
    exitosos = df[df[columna_resultado] == 'exitoso']
    no_exitosos = df[df[columna_resultado] == 'no_exitoso']
    
    min_count = min(len(exitosos), len(no_exitosos))
    
    exitosos_sample = exitosos.sample(min_count, random_state=42)
    no_exitosos_sample = no_exitosos.sample(min_count, random_state=42)
    
    return pd.concat([exitosos_sample, no_exitosos_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

df_balanced = balancear_dataframe_simple(df, 'resultado')
print(f"Datos balanceados: {df_balanced.shape}")
print(f"Distribución balanceada:\n{df_balanced['resultado'].value_counts()}")

Datos balanceados: (34818, 29)
Distribución balanceada:
resultado
exitoso       17409
no_exitoso    17409
Name: count, dtype: int64


In [8]:
categorical_columns = ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 
                       'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']

In [9]:
existing_categorical = [col for col in categorical_columns if col in df_balanced.columns]
print(f"Columnas categóricas disponibles: {existing_categorical}")

Columnas categóricas disponibles: ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']


In [10]:
df_processed = df_balanced.copy()

In [11]:
encoders = {}
for col in existing_categorical:
    print(f"Procesando {col}...")
    # Llenar valores nulos
    df_processed[col] = df_processed[col].fillna('unknown')
    
    # Codificar con LabelEncoder
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    encoders[col] = le
    
    print(f"  {col}: {len(le.classes_)} clases únicas")

Procesando cole_depto_ubicacion...
  cole_depto_ubicacion: 34 clases únicas
Procesando cole_jornada...
  cole_jornada: 6 clases únicas
Procesando cole_naturaleza...
  cole_naturaleza: 2 clases únicas
Procesando fami_educacionmadre...
  fami_educacionmadre: 11 clases únicas
Procesando fami_educacionpadre...
  fami_educacionpadre: 11 clases únicas
Procesando fami_estratovivienda...
  fami_estratovivienda: 6 clases únicas


In [12]:
feature_columns = existing_categorical
X = df_processed[feature_columns].copy()
y = df_processed['resultado'].map({'no_exitoso': 0, 'exitoso': 1})

print(f"\nFeatures finales: {X.shape}")
print(f"Columnas: {list(X.columns)}")


Features finales: (34818, 6)
Columnas: ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']


In [13]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X), 
    columns=X.columns, 
    index=X.index
)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nConjuntos de entrenamiento y prueba:")
print(f"  Train: {X_train.shape}, distribución: {np.bincount(y_train)}")
print(f"  Test: {X_test.shape}, distribución: {np.bincount(y_test)}")


Conjuntos de entrenamiento y prueba:
  Train: (27854, 6), distribución: [13927 13927]
  Test: (6964, 6), distribución: [3482 3482]


In [ ]:
improved_model = create_improved_model(X_train.shape[1])
improved_history, improved_probs = train_and_evaluate_model(
    improved_model, X_train, y_train, X_test, y_test, "Mejorado"
)
    
# Guardar el mejor modelo
best_model = improved_model
best_probs = improved_probs

c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando Baseline
Epoch 1/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8006 - loss: 0.4375 - val_accuracy: 0.8518 - val_loss: 0.3602 - learning_rate: 0.0100
Epoch 2/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8521 - loss: 0.3575 - val_accuracy: 0.8532 - val_loss: 0.3605 - learning_rate: 0.0100
Epoch 3/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8492 - loss: 0.3644 - val_accuracy: 0.8548 - val_loss: 0.3612 - learning_rate: 0.0100
Epoch 4/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8504 - loss: 0.3612 - val_accuracy: 0.8545 - val_loss: 0.3603 - learning_rate: 0.0100
Epoch 5/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8517 - loss: 0.3624 - val_accuracy: 0.8532 - val_loss: 0.3601 - learning_rate: 0.0100
Epoch 6/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8505 - loss: 0.3608 - val_accuracy: 0.8535 - val_loss: 0.3610 - learning_rate: 0.0100
Epoch 7/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accurac

c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


871/871 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8235 - loss: 0.4184 - val_accuracy: 0.8557 - val_loss: 0.3551 - learning_rate: 0.0010
Epoch 2/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8487 - loss: 0.3733 - val_accuracy: 0.8557 - val_loss: 0.3521 - learning_rate: 0.0010
Epoch 3/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8507 - loss: 0.3685 - val_accuracy: 0.8554 - val_loss: 0.3531 - learning_rate: 0.0010
Epoch 4/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8509 - loss: 0.3666 - val_accuracy: 0.8567 - val_loss: 0.3503 - learning_rate: 0.0010
Epoch 5/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8540 - loss: 0.3608 - val_accuracy: 0.8563 - val_loss: 0.3494 - learning_rate: 0.0010
Epoch 6/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8541 - loss: 0.3581 - val_accuracy: 0.8555 - val_loss: 0.3487 - learning_rate: 0.0010
Epoch 7/100
871/871 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8536 - loss: 0.3593 - val_a

In [19]:
best_model.save('modelo_final.keras')
# Guardar información de preprocesamiento
import pickle
preprocessing_info = {
    'feature_columns': feature_columns,
    'encoders': encoders,
    'scaler': scaler,
    'categorical_columns': existing_categorical
    }
    
with open('preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

In [17]:
def test_predictions_on_sample(model_path, preprocessing_path, sample_data):
    """
    Probar predicciones en datos de muestra
    """
    model = tf.keras.models.load_model(model_path)
    
    with open(preprocessing_path, 'rb') as f:
        prep_info = pickle.load(f)
    
    # Procesar datos
    sample_processed = sample_data.copy()
    
    for col in prep_info['categorical_columns']:
        if col in sample_processed.columns:
            sample_processed[col] = sample_processed[col].fillna('unknown')
            le = prep_info['encoders'][col]
            # Manejar valores no vistos
            sample_processed[col] = sample_processed[col].astype(str).apply(
                lambda x: x if x in le.classes_ else le.classes_[0]
            )
            sample_processed[col] = le.transform(sample_processed[col])
    
    # Normalizar
    sample_scaled = prep_info['scaler'].transform(sample_processed[prep_info['feature_columns']])
    
    # Predecir
    probs = model.predict(sample_scaled, verbose=0)
    preds = (probs > 0.5).astype(int)
    
    return probs, preds

In [22]:
sample_data = df_balanced[existing_categorical].iloc[:-1]
    
try:
    probs, preds = test_predictions_on_sample(
        'modelo_final.keras',
        'preprocessing_info.pkl',
        sample_data
        )
        
    print("Predicciones en muestra:")
    for i, (prob, pred) in enumerate(zip(probs.flatten(), preds.flatten())):
        real = df_balanced['resultado'].iloc[i]
        pred_label = 'exitoso' if pred == 1 else 'no_exitoso'
        print(f"  Ejemplo {i+1}: {pred_label} (prob: {prob:.4f}), Real: {real}")
            
except Exception as e:
    print(f"Error al probar predicciones: {e}")

Predicciones en muestra:
  Ejemplo 1: no_exitoso (prob: 0.2332), Real: exitoso
  Ejemplo 2: no_exitoso (prob: 0.1587), Real: no_exitoso
  Ejemplo 3: exitoso (prob: 0.7596), Real: no_exitoso
  Ejemplo 4: no_exitoso (prob: 0.4434), Real: no_exitoso
  Ejemplo 5: exitoso (prob: 0.9924), Real: exitoso
  Ejemplo 6: exitoso (prob: 0.9642), Real: exitoso
  Ejemplo 7: no_exitoso (prob: 0.1587), Real: no_exitoso
  Ejemplo 8: exitoso (prob: 0.6810), Real: exitoso
  Ejemplo 9: exitoso (prob: 0.9872), Real: exitoso
  Ejemplo 10: exitoso (prob: 0.9935), Real: exitoso
  Ejemplo 11: no_exitoso (prob: 0.2009), Real: no_exitoso
  Ejemplo 12: no_exitoso (prob: 0.1015), Real: no_exitoso
  Ejemplo 13: no_exitoso (prob: 0.1139), Real: no_exitoso
  Ejemplo 14: no_exitoso (prob: 0.0220), Real: no_exitoso
  Ejemplo 15: no_exitoso (prob: 0.1638), Real: no_exitoso
  Ejemplo 16: no_exitoso (prob: 0.0295), Real: no_exitoso
  Ejemplo 17: exitoso (prob: 0.8379), Real: exitoso
  Ejemplo 18: exitoso (prob: 0.8379), Re

In [26]:
df_balanced[categorical_columns].iloc[18:19]

,cole_depto_ubicacion,cole_jornada,cole_naturaleza,fami_educacionmadre,fami_educacionpadre,fami_estratovivienda
18,BOLIVAR,MAÑANA,NO OFICIAL,Educación profesional completa,Educación profesional completa,Estrato 3
